In [ ]:
from appgeopy import *
from my_packages import *

# ------------------------------------------------------------------------------
# Configuration File Path
# ------------------------------------------------------------------------------
configure_fpath = r"D:\VENV_PYTHON\insar_scriptflow\B__post_insstackpsi_config.ini"

# ------------------------------------------------------------------------------
# Read and Parse the Configuration File
# ------------------------------------------------------------------------------
config = configparser.ConfigParser()
config.read(configure_fpath)
print(f"Configuration file path: {configure_fpath}")

# ------------------------------------------------------------------------------
# Extract General Parameters from the Configuration File
# ------------------------------------------------------------------------------
# Main folder containing the INSSTACKPSI data
mainfolder = config["GENERAL"]["insstackpsi_mainfolder"]

# Paths for INSSTACKPSI and post-processing folders
insstackpsi_fld = os.path.join(mainfolder, "13_INSSTACKPSI_inspsn")
post_insstackpsi_fld = os.path.join(mainfolder, "14_POST-PROCESSING")

# Path to the displacement data pickle file
o_displacement_fpath = os.path.join(post_insstackpsi_fld, "ras2pnt_MODIFIED_oChoushui_DISPLACEMENT_LOS_mm.pkl")

# Path to the GPS LOS difference data file
gps_los_difference_fpath = config["GPS_DIFFERENCE"]["gps_los_difference_fpath"]

# Buffer radius for searching nearby points
radius_of_buffer = int(config["PCS_SEARCH"]["buffer_radius"])

# Output file name for LOS difference results
los_difference_savename = config["GPS_DIFFERENCE"]["los_difference_output"]

# Path to the input M-file containing datetime information
input_mfile = config["GENERAL"]["input_mfile"]

# ------------------------------------------------------------------------------
# Load Displacement Data and Configuration
# ------------------------------------------------------------------------------
# Load the displacement data into a GeoDataFrame
o_displacement_geodf = pd.read_pickle(o_displacement_fpath, compression="zip")
o_displacement_geodf = o_displacement_geodf.set_index("PointKey")

# Extract datetime information from the input M-file
mfile_datetimes = datetime_handle.extract_datetime_from_mfile(input_mfile)[1:]

# Load selected point keys based on the buffer radius
select_pointkey_fpath = glob(os.path.join(post_insstackpsi_fld, f"*{radius_of_buffer}*.pkl"))[0]
select_pointkey = pd.read_pickle(select_pointkey_fpath, compression="zip")

# Get the unique GPS stations from the selection
available_stations = select_pointkey["STATION"].unique()

# Construct the save path for LOS difference results
los_difference_savepath = os.path.join(post_insstackpsi_fld, los_difference_savename)

# Remove the existing LOS difference file if it exists
if os.path.exists(los_difference_savepath):
    os.remove(los_difference_savepath)

# ------------------------------------------------------------------------------
# Process Each GPS Station and Compare LOS Displacements
# ------------------------------------------------------------------------------
for select_station in tqdm(available_stations):
    # Filter PointKeys associated with the current station
    pointkey_by_station = select_pointkey.loc[select_pointkey["STATION"] == select_station, "PointKey"]

    # Extract displacement data for the current station
    psc_by_station = o_displacement_geodf.loc[pointkey_by_station, mfile_datetimes]

    # Calculate the average displacement over all PS points for the current station
    avg_deform_psc_by_station = pd.DataFrame(psc_by_station.mean(axis=0), columns=["LOS(mm)"])
    avg_deform_psc_by_station.index = avg_deform_psc_by_station.index.map(datetime_handle.convert_to_datetime)

    # Load GPS LOS difference data for the current station
    gps_los_difference_by_station = pd.read_excel(
        gps_los_difference_fpath,
        sheet_name=select_station,
        usecols=[0, 4],  # Assume the first column is date and the fifth column is LOS difference
        parse_dates=[0],  # Parse the first column as dates
        index_col=[0]  # Set the first column as the index
    )

    # Find the common dates between PS and GPS data
    mutual_index = datetime_handle.intersect_time_index(
        df1_index=avg_deform_psc_by_station.index,
        df2_index=gps_los_difference_by_station.index
    )

    # Compare LOS displacements between PS and GPS data over the common dates
    los_difference = ts_disp.compare_LOS_disp(
        psc_df=avg_deform_psc_by_station,
        gps_df=gps_los_difference_by_station,
        mutual_index=mutual_index
    )

    # Reset index to prepare for saving
    los_difference = los_difference.reset_index(drop=False)

    # Save the LOS difference data to the Excel file
    data_io.save_df_to_excel(
        df_to_save=los_difference,
        filepath=los_difference_savepath,
        sheet_name=select_station,
        verbose=False
    )

print("Processing completed successfully.")